In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
serpapi_api_key = os.environ["SERPAPI_API_KEY"]


In [5]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

In [6]:
llm_model = OpenAI()
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an 'answer' key that answera the following question:\
    {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain =  json_prompt | llm_model | json_parser

In [10]:
response = json_chain.invoke({"question": "what is the biggest country?"})
print(response)
type(response)

{'answer': 'Russia'}


dict

se puede usar Pydantic para personalizar el formato

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [22]:
# Define a Pydantic object with the desired output format
class Joke(BaseModel):
    setup: str = Field(description="question to setup a joke")
    punchline: str = Field(description="answer to resolve the joke")

#Define the parser referring the Pydantic object
parser = SimpleJsonOutputParser(pydantic_object=Joke)

# Define the parser format instructions in the prompt definition
prompt = PromptTemplate(
    template = "Answer the user query, \n {format_instructions}\n{query}",
    input_variables= ["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
joke_chain = prompt | llm_model | parser
joke_chain.invoke({"query":"tell me a joke about Donald Trump"})

{'setup': 'Why did Donald Trump go to the bank?',
 'punchline': 'To get his Cabinet!'}